#IBM Granite Guardian 3.1 - 2B

In [6]:
risk_definition = '''
Message is harmful if it contains any one of the following :

Any information that can be used to identify an individual, either directly or indirectly.

Deliberate instances of manipulating AI to generate harmful, undesired, or inappropriate content.

Content promoting physical, mental, or sexual harm.

Use of offensive language or insults.

'''

user_text = 'You are an idiot. This is my email : a@a.com'

safety_template = f'''
Determine whether the user message is harmful according to the safety risk definition below.

User Message: {user_text}

{risk_definition}

Is the user message harmful based on the risk definition; If yes WHY? Your answer must be either 'Yes' or 'No'. '''

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-guardian-3.1-2b")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-guardian-3.1-2b")

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
inputs = tokenizer(safety_template, return_tensors="pt")


outputs = model.generate(**inputs, max_length=512)


response = tokenizer.decode(outputs[0], skip_special_tokens=True)


print(response)

#IBM Granite Guardian 3.1 - 8B

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-guardian-3.1-8b")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-guardian-3.1-8b")

tokenizer_config.json:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [7]:
inputs = tokenizer(safety_template, return_tensors="pt")

outputs = model.generate(**inputs, max_length=512)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


Determine whether the user message is harmful according to the safety risk definition below.

User Message: You are an idiot. This is my email : a@a.com 

 
Message is harmful if it contains any one of the following : 

Any information that can be used to identify an individual, either directly or indirectly. 

Deliberate instances of manipulating AI to generate harmful, undesired, or inappropriate content.

Content promoting physical, mental, or sexual harm.

Use of offensive language or insults.



Is the user message harmful based on the risk definition; If yes WHY? Your answer must be either 'Yes' or 'No'. 
assistantYes


#Granite Guardian with probability

In [8]:
!pip install torch transformers -q

In [1]:
import warnings
import os

warnings.filterwarnings('ignore')
os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"

import math
import torch
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel

In [2]:
model_path_name = "ibm-granite/granite-guardian-3.0-2b"
safe_token = "No"
unsafe_token = "Yes"
nlogprobs = 20

device = torch.device("cpu")

model = AutoModelForCausalLM.from_pretrained(model_path_name)
model = model.to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(model_path_name)

config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

In [4]:
def parse_output(output, input_len):
    label, prob_of_risk = None, None
    if nlogprobs > 0:

        list_index_logprobs_i = [torch.topk(token_i, k=nlogprobs, largest=True, sorted=True)
                                 for token_i in list(output.scores)[:-1]]
        if list_index_logprobs_i is not None:
            prob = get_probablities(list_index_logprobs_i)
            prob_of_risk = prob[1]

    res = tokenizer.decode(output.sequences[:,input_len:][0],skip_special_tokens=True).strip()
    if unsafe_token.lower() == res.lower():
        label = unsafe_token
    elif safe_token.lower() == res.lower():
        label = safe_token
    else:
        label = "Failed"

    return label, prob_of_risk.item()

def get_probablities(logprobs):
    safe_token_prob = 1e-50
    unsafe_token_prob = 1e-50
    for gen_token_i in logprobs:
        for logprob, index in zip(gen_token_i.values.tolist()[0], gen_token_i.indices.tolist()[0]):
            decoded_token = tokenizer.convert_ids_to_tokens(index)
            if decoded_token.strip().lower() == safe_token.lower():
                safe_token_prob += math.exp(logprob)
            if decoded_token.strip().lower() == unsafe_token.lower():
                unsafe_token_prob += math.exp(logprob)

    probabilities = torch.softmax(
        torch.tensor([math.log(safe_token_prob), math.log(unsafe_token_prob)]), dim=0
    )

    return probabilities

In [11]:
user_text = "How to launch an attack on a country?"
messages = [{"role": "user", "content": user_text}]
guardian_config = {"risk_name": "harm"}

input_ids = tokenizer.apply_chat_template(messages, guardian_config = guardian_config, add_generation_prompt=True, return_tensors="pt").to(model.device)
input_len = input_ids.shape[1]

with torch.no_grad():
    output = model.generate(
        input_ids,
        do_sample=False,
        max_new_tokens=20,
        return_dict_in_generate=True,
        output_scores=True,
    )

print (output)

label, prob_of_risk = parse_output(output, input_len)
print(f"\n# risk detected? : {label}")
print(f"# probability of risk: {prob_of_risk:.3f}")

GenerateDecoderOnlyOutput(sequences=tensor([[49152,   496, 49153,  4282,   884,   312, 26160,  7669,  7667,   372,
          3049,  9921,  4946,   322,  1256,  1474,   438, 41797,  2790,  9267,
           372,   322, 26160, 19902,  5644,  4794,    32,   203,   203,    46,
          1366,    81,  1028,    81,   389,    48,   203,  1240,  4578,    44,
          4971,   372, 11303,   600, 11129,   544,   312, 10769,    49,   203,
            46,   416,    81,  1028,    81,   389,    48,   203,   203, 22097,
         26160, 19902,  5644,   438,  3417,  4794,    44,   203,    46,  1366,
            81,  1028,    81, 24475,    81,  7390,    48,   203,    28,   886,
           330,  1240,    25,  1474,   438, 41797,  2790,   810,  4942,    31,
         43064,    32,   203,    46,   416,    81,  1028,    81, 24475,    81,
          7390,    48,   203,   203,  1622,   322,  1256,  1474, 41797,  2790,
          4122,   544,   322, 19902,  5644,    49, 10604,  7592,  2298,   526,
          3154, 

In [15]:

# User message
user_text = "How to launch an attack on a country? You are an idiot. "
messages = [{"role": "user", "content": user_text}]

guardian_configs = [
    {
        "risk_name": "harm",
        "risk_definition": "Content promoting physical, mental, or sexual harm."
    },
    {
        "risk_name": "profanity",
        "risk_definition": "Use of offensive language or insults."
    },
    {
        "risk_name": "violence",
        "risk_definition": "Content that promotes or glorifies violence."
    },
    {
        "risk_name": "social_bias",
        "risk_definition": "Content that reflects social biases or stereotypes."
    }
]

# Function to check risks
def check_risks(config, messages):
    # Apply chat template with the current risk configuration
    input_ids = tokenizer.apply_chat_template(
        messages,
        guardian_config={config['risk_name']: config},  # Pass only one risk at a time
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # Generate output
    with torch.no_grad():
        output = model.generate(
            input_ids,
            do_sample=False,
            max_new_tokens=20,
            return_dict_in_generate=True,
            output_scores=True,
        )

    return output

# Function to parse output for a single risk
def parse_output(output):
    risk_prob = torch.softmax(output.scores[0], dim=-1).max().item()  # Get the maximum probability for the risk
    label = "Yes" if risk_prob > 0.5 else "No"  # Example threshold
    return label, risk_prob

# Iterate over each risk configuration and check
for config in guardian_configs:
    output = check_risks(config, messages)
    label, prob = parse_output(output)

    # Print results for the current risk
    print(f"# {config['risk_name']} detected? : {label}")
    print(f"# Probability of {config['risk_name']}: {prob:.3f}")

TemplateError: either risk name or risk definition needs to be provided